## Reading Web Server logs using Spark Structured Streaming

As we are ready with the ability to simulate log message generation, let us get into reading these logs using Spark Structured Streaming.
* `spark` which is of type `sparkSession` have an attribute called as `readStream`. It is of type `pyspark.sql.streaming.DataStreamReader`.
* It exposes APIs such as `csv`, `json`, etc along with `format`. To read data from web servers, we can use `socket` as format.
* We need to set options `host` and `port`, then invoke `load` to read data in streaming fashion.
* It will create an object which will be of type `pyspark.sql.dataframe.DataFrame`.

In [0]:
spark.readStream

Out[4]: <pyspark.sql.streaming.DataStreamReader at 0x7fe08040af70>

In [0]:
socketDF = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9000) \
    .load()

In [0]:
socketDF.isStreaming

Out[6]: True

In [0]:
socketDF.printSchema()

root
-- value: string (nullable = true)

In [0]:
socketDF.show() # throws exceptions

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-4003030508660554> in <module> 
 ----> 1 socketDF . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 488 """
 489 if isinstance ( truncate , bool ) and truncate : 
 --> 490 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 491 else : 
 492 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Queries with streaming sources must be executed with writeStream.start();
socket

* If no trigger setting is explicitly specified, then by default, the query will be executed in micro-batch mode, where micro-batches will be generated as soon as the previous micro-batch has completed processing.

In [0]:
socketDF.writeStream.

Out[9]: <pyspark.sql.streaming.DataStreamWriter at 0x7fe0804f9dc0>

In [0]:
socketDF \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

Out[13]: <pyspark.sql.streaming.StreamingQuery at 0x7fe077cf90d0>

* Run the below code and watch the output. You will see messages being processed every 5 seconds.

In [0]:
socketDF \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()

# Triggers every 5 seconds

Out[12]: <pyspark.sql.streaming.StreamingQuery at 0x7fe077cf94f0>